In [1]:
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten,Conv2D,Conv1D,MaxPooling1D,  Dropout
from keras.layers import concatenate
from keras.layers import Bidirectional, GlobalMaxPool1D,BatchNormalization
from keras.models import Model, Sequential
from keras.layers import Convolution1D,GlobalMaxPooling1D
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.utils import to_categorical
import numpy as np
import sys
import os

/Users/harvey/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#df = pd.read_csv("C:/Users/Harvey/Desktop/Yelp_data_set/restuarant_review_5_label_unbalanced.csv")
df = pd.read_csv("/Users/harvey/Desktop/Data/restuarant_review_5_label_unbalanced.csv")
#df = pd.read_csv("/home/ec2-user/Data/restuarant_review_5_label_unbalanced.csv")
train = df.sample(frac = 0.8,random_state = 200)
test = df.drop(train.index)
train.loc[:,'stars'] -= 1
print(np.unique(train['stars']))

[0 1 2 3 4]


In [3]:
max_features = 6000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(train['Processed_Reviews'])
list_tokenized_train = tokenizer.texts_to_sequences(train['Processed_Reviews'])


maxlen = 130
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
y = to_categorical(train['stars'])
#####################
# Test data
tokenizer.fit_on_texts(test['Processed_Reviews'])
list_tokenized_test = tokenizer.texts_to_sequences(test['Processed_Reviews'])


maxlen = 130
X_test = pad_sequences(list_tokenized_test, maxlen=maxlen)
test.loc[:,'stars'] -=1
y_test = test['stars']
print(np.unique(test['stars']))

[0 1 2 3 4]


In [4]:
#####################################################################
# Using pretrained glove vector
#####################################################################
#GLOVE_DIR = "/home/ec2-user/Data/"
GLOVE_DIR = "/Users/harvey/Desktop/Data/"
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'),encoding = 'utf8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Total %s word vectors in Glove 6B 100d.' % len(embeddings_index))

Total 400000 word vectors in Glove 6B 100d.


In [5]:
word_index = tokenizer.word_index
embed_size = 100
embedding_matrix = np.random.random((len(word_index) + 1, embed_size))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
        
embedding_layer = Embedding(len(word_index) + 1,
                            embed_size,
                            weights=[embedding_matrix],
                            input_length=maxlen,
                            trainable=False)

In [6]:
############
#Test to be deleted

In [9]:


#Randomly initialized 
embedding_layer = Embedding(len(word_index) + 1,
                            embed_size,
                            input_length=maxlen,
                            trainable=True)


#############################################
# Original yoon kim with batch norm and drop out0
#############################################
conv_filters = 128
sequence_input = Input(shape=(maxlen,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

# Specify each convolution layer and their kernel siz i.e. n-grams 
conv1_1 = Conv1D(filters=conv_filters, kernel_size=3,kernel_regularizer=regularizers.l2(0.1))(embedded_sequences)
btch1_1 = BatchNormalization()(conv1_1)
actv1_1 = Activation('relu')(btch1_1)
drp1_1  = Dropout(0.2)(actv1_1)
glmp1_1 = MaxPooling1D(pool_size = 4)(drp1_1)
#glmp1_1 = GlobalMaxPool1D()(drp1_1)

conv1_2 = Conv1D(filters=conv_filters, kernel_size=4,kernel_regularizer=regularizers.l2(0.1))(embedded_sequences)
btch1_2 = BatchNormalization()(conv1_2)
actv1_2 = Activation('relu')(btch1_2)
drp1_2  = Dropout(0.2)(actv1_2)
glmp1_2 = MaxPooling1D(pool_size = 4)(drp1_2)
#glmp1_2 =  GlobalMaxPool1D()(drp1_2)



conv1_3 = Conv1D(filters=conv_filters, kernel_size=5,kernel_regularizer=regularizers.l2(0.1))(embedded_sequences)
btch1_3 = BatchNormalization()(conv1_3)
actv1_3 = Activation('relu')(btch1_3)
drp1_3  = Dropout(0.2)(actv1_3)
glmp1_3 = MaxPooling1D(pool_size = 4)(drp1_3)
#glmp1_3 = GlobalMaxPool1D()(drp1_3)

conv1_4 = Conv1D(filters=conv_filters, kernel_size=6,kernel_regularizer=regularizers.l2(0.1))(embedded_sequences)
btch1_4 = BatchNormalization()(conv1_4)
actv1_4 = Activation('relu')(btch1_4)
drp1_4  = Dropout(0.2)(actv1_4)
glmp1_4 = MaxPooling1D(pool_size = 4)(drp1_4)
#glmp1_4 = GlobalMaxPool1D()(drp1_4)

# Gather all convolution layers
cnct = concatenate([glmp1_1, glmp1_2, glmp1_3, glmp1_4], axis=1)
drp1 = Dropout(0.2)(cnct)

dns1  = Dense(256, activation='relu')(drp1)
btch1 = BatchNormalization()(dns1)
drp2  = Dropout(0.2)(btch1)
flat = Flatten()(drp2)
out = Dense(5, activation='softmax')(flat)


model = Model(inputs=sequence_input, outputs=out)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model.summary()


batch_size = 512
epochs = 100
history = model.fit(X_t,y, batch_size=batch_size, epochs=epochs, validation_split=0.2)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 130)          0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 130, 100)     27500700    input_3[0][0]                    
__________________________________________________________________________________________________
conv1d_9 (Conv1D)               (None, 128, 128)     38528       embedding_4[0][0]                
__________________________________________________________________________________________________
conv1d_10 (Conv1D)              (None, 127, 128)     51328       embedding_4[0][0]                
__________________________________________________________________________________________________
conv1d_11 

KeyboardInterrupt: 

In [ ]:
# Test to be deleted
##################

In [6]:
model = Sequential()
model.add(embedding_layer)
model.add(Conv1D(filters = 50, kernel_size = 2, padding='same',activation = "relu"))
model.add(GlobalMaxPool1D())
#model.add(Flatten())
model.add(Dense(5, activation="softmax"))
model.summary()
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 130, 100)          27500700  
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 130, 50)           10050     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 255       
Total params: 27,511,005
Trainable params: 10,305
Non-trainable params: 27,500,700
_________________________________________________________________


In [7]:
batch_size = 100
epochs = 10
model.fit(X_t,y, batch_size=batch_size, epochs=epochs, validation_split=0.2)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 377894 samples, validate on 94474 samples
Epoch 1/10
377894/377894 [==============================] - 20s 52us/step - loss: 1.0487 - acc: 0.5503 - val_loss: 0.9574 - val_acc: 0.5866
Epoch 2/10
377894/377894 [==============================] - 18s 48us/step - loss: 0.9377 - acc: 0.5945 - val_loss: 0.9233 - val_acc: 0.5996
Epoch 3/10
377894/377894 [==============================] - 18s 48us/step - loss: 0.9121 - acc: 0.6041 - val_loss: 0.9144 - val_acc: 0.6029
Epoch 4/10
377894/377894 [==============================] - 18s 48us/step - loss: 0.8992 - acc: 0.6096 - val_loss: 0.9089 - val_acc: 0.6030
Epoch 5/10
377894/377894 [==============================] - 18s 48us/step - loss: 0.8911 - acc: 0.6122 - val_loss: 0.8974 - val_acc: 0.6108
Epoch 6/10
377894/377894 [==============================] - 18s 48us/step - loss: 0.8852 - acc: 0.6137 - val_loss: 0.9037 -

In [8]:
prediction = model.predict(X_test)
y_pred = np.argmax(prediction,axis = 1)
y_test = np.array(y_test)
print(y_pred[:10])
print(y_test.shape)
print(y_pred.shape)
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, confusion_matrix
print('accuracy :{0}'.format(accuracy_score(y_pred, y_test)))

[3 3 4 3 4 3 0 3 3 2]
(118092,)
(118092,)
accuracy :0.4749940724181147


In [9]:
embed_size = 100
embedding_layer = Embedding(len(word_index) + 1,
                            embed_size,
                            weights=[embedding_matrix],
                            input_length=maxlen,
                            trainable=False)

In [10]:
#############################################
# Original yoon kim with batch norm and drop out0
#############################################
conv_filters = 128
sequence_input = Input(shape=(maxlen,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

# Specify each convolution layer and their kernel siz i.e. n-grams 
conv1_1 = Conv1D(filters=conv_filters, kernel_size=3)(embedded_sequences)
btch1_1 = BatchNormalization()(conv1_1)
actv1_1 = Activation('relu')(btch1_1)
drp1_1  = Dropout(0.2)(actv1_1)
glmp1_1 = GlobalMaxPooling1D()(drp1_1)

conv1_2 = Conv1D(filters=conv_filters, kernel_size=4)(embedded_sequences)
btch1_2 = BatchNormalization()(conv1_2)
actv1_2 = Activation('relu')(btch1_2)
drp1_2  = Dropout(0.2)(actv1_2)
glmp1_2 = GlobalMaxPooling1D()(drp1_2)

conv1_3 = Conv1D(filters=conv_filters, kernel_size=5)(embedded_sequences)
btch1_3 = BatchNormalization()(conv1_3)
actv1_3 = Activation('relu')(btch1_3)
drp1_3  = Dropout(0.2)(actv1_3)
glmp1_3 = GlobalMaxPooling1D()(drp1_3)

conv1_4 = Conv1D(filters=conv_filters, kernel_size=6)(embedded_sequences)
btch1_4 = BatchNormalization()(conv1_4)
actv1_4 = Activation('relu')(btch1_4)
drp1_4  = Dropout(0.2)(actv1_4)
glmp1_4 = GlobalMaxPooling1D()(drp1_4)

# Gather all convolution layers
cnct = concatenate([glmp1_1, glmp1_2, glmp1_3, glmp1_4], axis=1)
drp1 = Dropout(0.2)(cnct)

dns1  = Dense(32, activation='relu')(drp1)
btch1 = BatchNormalization()(dns1)
drp2  = Dropout(0.2)(btch1)

out = Dense(5, activation='softmax')(drp2)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [11]:
model = Model(inputs=sequence_input, outputs=out)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 130)          0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 130, 100)     27500700    input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 128, 128)     38528       embedding_2[0][0]                
__________________________________________________________________________________________________
conv1d_3 (Conv1D)               (None, 127, 128)     51328       embedding_2[0][0]                
__________________________________________________________________________________________________
conv1d_4 (

In [12]:
batch_size = 100
epochs = 10
model.fit(X_t,y, batch_size=batch_size, epochs=epochs, validation_split=0.2)

Train on 377894 samples, validate on 94474 samples
Epoch 1/10
377894/377894 [==============================] - 127s 335us/step - loss: 1.0949 - acc: 0.5313 - val_loss: 0.9862 - val_acc: 0.5708
Epoch 2/10
377894/377894 [==============================] - 124s 327us/step - loss: 0.9723 - acc: 0.5818 - val_loss: 0.9225 - val_acc: 0.5969
Epoch 3/10
377894/377894 [==============================] - 124s 328us/step - loss: 0.9368 - acc: 0.5960 - val_loss: 0.8970 - val_acc: 0.6121
Epoch 4/10
377894/377894 [==============================] - 124s 327us/step - loss: 0.9153 - acc: 0.6051 - val_loss: 0.8880 - val_acc: 0.6111
Epoch 5/10
377894/377894 [==============================] - 124s 327us/step - loss: 0.9005 - acc: 0.6122 - val_loss: 0.8794 - val_acc: 0.6158
Epoch 6/10
377894/377894 [==============================] - 123s 325us/step - loss: 0.8871 - acc: 0.6177 - val_loss: 0.8638 - val_acc: 0.6261
Epoch 7/10
377894/377894 [==============================] - 123s 327us/step - loss: 0.8769 - acc:

In [13]:
prediction = model.predict(X_test)
y_pred = np.argmax(prediction,axis = 1)
y_test = np.array(y_test)
print(y_pred[:10])
print(y_test.shape)
print(y_pred.shape)
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, confusion_matrix
print('accuracy :{0}'.format(accuracy_score(y_pred, y_test)))

[3 2 4 3 4 4 4 3 3 2]
(118092,)
(118092,)
accuracy :0.5039206720184263


In [14]:
##############################
# Orignal Yoon Kim 
##############################
#############################################

#############################################
conv_filters = 128
sequence_input = Input(shape=(maxlen,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

# Specify each convolution layer and their kernel siz i.e. n-grams 
conv1_1 = Conv1D(filters=conv_filters, kernel_size=3)(embedded_sequences)

actv1_1 = Activation('relu')(conv1_1)
glmp1_1 = GlobalMaxPooling1D()(actv1_1)

conv1_2 = Conv1D(filters=conv_filters, kernel_size=4)(embedded_sequences)
actv1_2 = Activation('relu')(conv1_2)
glmp1_2 = GlobalMaxPooling1D()(actv1_2)

conv1_3 = Conv1D(filters=conv_filters, kernel_size=5)(embedded_sequences)
actv1_3 = Activation('relu')(conv1_3)
glmp1_3 = GlobalMaxPooling1D()(actv1_3)

conv1_4 = Conv1D(filters=conv_filters, kernel_size=6)(embedded_sequences)
actv1_4 = Activation('relu')(conv1_4)
glmp1_4 = GlobalMaxPooling1D()(actv1_4)

# Gather all convolution layers
cnct = concatenate([glmp1_1, glmp1_2, glmp1_3, glmp1_4], axis=1)

dns1  = Dense(32, activation='relu')(cnct)
drp2  = Dropout(0.5)(dns1)

out = Dense(5, activation='softmax')(drp2)

In [15]:
model = Model(inputs=sequence_input, outputs=out)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 130)          0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 130, 100)     27500700    input_2[0][0]                    
__________________________________________________________________________________________________
conv1d_6 (Conv1D)               (None, 128, 128)     38528       embedding_2[1][0]                
__________________________________________________________________________________________________
conv1d_7 (Conv1D)               (None, 127, 128)     51328       embedding_2[1][0]                
__________________________________________________________________________________________________
conv1d_8 (

In [16]:
batch_size = 100
epochs = 10
model.fit(X_t,y, batch_size=batch_size, epochs=epochs, validation_split=0.2)

Train on 377894 samples, validate on 94474 samples
Epoch 1/10
377894/377894 [==============================] - 69s 183us/step - loss: 1.4620 - acc: 0.4017 - val_loss: 1.4494 - val_acc: 0.4002
Epoch 2/10
377894/377894 [==============================] - 68s 180us/step - loss: 1.4500 - acc: 0.4021 - val_loss: 1.4496 - val_acc: 0.4002
Epoch 3/10
377894/377894 [==============================] - 68s 181us/step - loss: 1.4499 - acc: 0.4020 - val_loss: 1.4494 - val_acc: 0.4002
Epoch 4/10
377894/377894 [==============================] - 68s 180us/step - loss: 1.4497 - acc: 0.4021 - val_loss: 1.4495 - val_acc: 0.4002
Epoch 5/10
377894/377894 [==============================] - 68s 180us/step - loss: 1.4497 - acc: 0.4021 - val_loss: 1.4494 - val_acc: 0.4002
Epoch 6/10
377894/377894 [==============================] - 68s 181us/step - loss: 1.4499 - acc: 0.4020 - val_loss: 1.4496 - val_acc: 0.4001
Epoch 7/10
377894/377894 [==============================] - 68s 180us/step - loss: 1.4498 - acc: 0.4020

In [18]:
prediction = model.predict(X_test)
y_pred = np.argmax(prediction,axis = 1)
y_test = np.array(y_test)
print(y_pred[:10])
print(y_test.shape)
print(y_pred.shape)
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, confusion_matrix
print('accuracy :{0}'.format(accuracy_score(y_pred, y_test)))

[4 0 4 4 4 4 4 4 4 0]
(118092,)
(118092,)
accuracy :0.45113132134268197
